In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_excel('Gallery Raw Data 2019_01(SL).xlsx',sheet_name =3)#利用read_excel 获取文档，sheetname是相关的sheet
data["Year/Month"] = data["Year/Month"].astype('str')#强行把时间改成字符串格式

amlist = pd.read_excel('amlist.xlsx',sheet_name=0,header=None)
filter = amlist.values

In [3]:
def cleanData(filter):
    rawData = data.loc[data["Advertiser Name"]==filter[0]]#将某一个客户名的数据筛选出来
    return rawData

In [4]:
#indexA是Initiative by month
def indexA():
    indexA=["Line Item ID","Line Item Duration","Designer Name","Year/Month"]
    return indexA

def SeriesMakerSum(rawData,index):
    Grouped = rawData.groupby(index)#根据index的要求使用Groupby函数制作一个数据透视表
    Group = Grouped.sum()#Groupby 过程中相同index的项采用sum方法加在一块显示
    PI = Group["Product Impressions"]
    CTP = Group["Click Thru to Product"]
    AF = Group["Add to Favorites"]
    CTASW = Group["Click Through to Advertiser Site from Web"]
    CMS = Group["Click to Marketplace Storefront"]
    FDE = Group["Filter by Designer Events"]
    return FDE,PI,CTP,AF,CTASW,CMS
def SeriesMakerMax(rawData,index):
    Grouped = rawData.groupby(index)#根据index的要求使用Groupby函数制作一个数据透视表
    Group = Grouped.sum()#Groupby 过程中相同index的项采用sum方法加在一块显示
    LO=Group["# of Live Objects"]
    return LO

def galleryDesignerTotal(PI,CTP,AF,CTASW,CMS,FDE):
     # 名字来源：liveOjbect_DesignerTotal 依此类推
    PI_DT = PI.groupby(["Line Item ID","Line Item Duration","Designer Name"]).sum()
    CTP_DT = CTP.groupby(["Line Item ID","Line Item Duration","Designer Name"]).sum()
    AF_DT = AF.groupby(["Line Item ID","Line Item Duration","Designer Name"]).sum()
    CTASW_DT = CTASW.groupby(["Line Item ID","Line Item Duration","Designer Name"]).sum()
    CMS_DT = CMS.groupby(["Line Item ID","Line Item Duration","Designer Name"]).sum()
    FDE_DT = FDE.groupby(["Line Item ID","Line Item Duration","Designer Name"]).sum()
    return FDE_DT,PI_DT,CTP_DT,AF_DT,CTASW_DT,CMS_DT

def galleryLineItemTotal(PI,CTP,AF,CTASW,CMS,FDE):
    FDE_LIT = FDE.groupby(["Line Item ID","Line Item Duration"]).sum() # 名字来源：liveOjbect_LineItemTotal 依此类推
    PI_LIT = PI.groupby(["Line Item ID","Line Item Duration"]).sum()
    CTP_LIT = CTP.groupby(["Line Item ID","Line Item Duration"]).sum()
    AF_LIT = AF.groupby(["Line Item ID","Line Item Duration"]).sum()
    CTASW_LIT = CTASW.groupby(["Line Item ID","Line Item Duration"]).sum()
    CMS_LIT = CMS.groupby(["Line Item ID","Line Item Duration"]).sum()
    CMS_LIT = CMS.groupby(["Line Item ID","Line Item Duration"]).sum()
    return FDE_LIT,PI_LIT,CTP_LIT,AF_LIT,CTASW_LIT,CMS_LIT
def galleryTotal(FDE,PI,CTP,AF,CTASW,CMS):
    FDE_T = FDE.sum() # 名字来源：liveOjbect_LineItemTotal 依此类推
    PI_T = PI.sum()
    CTP_T = CTP.sum()
    AF_T = AF.sum()
    CTASW_T = CTASW.sum()
    CMS_T = CMS.sum()
    return FDE_T,PI_T,CTP_T,AF_T,CTASW_T,CMS_T
def galleryMax(LO):
    LO_DM = LO.groupby(["Line Item ID","Line Item Duration","Designer Name"]).max()
    LO_LIM = LO_DM.groupby(["Line Item ID","Line Item Duration"])
    LO_LIM = LO_LIM.sum()
    LO_T = LO_LIM.sum()
    return LO_DM,LO_LIM,LO_T
def count(FDE):
    count_LI = np.array(FDE.index.labels[1])#利用Series的index 的labels进行分类计算有多少个drop_initiative，然后进行转化为数组类型
    #但是由于此时的count是[0,0,0,1,1,1,2,2,2]的样子，需要去重复，采用set的方法实现一个无序不重复的对象
    count_LI = list(set(count_LI))#然后再list转化为list
    count_D = np.array(FDE.index.labels[2])
    count_D = list(set(count_D))
    return count_LI,count_D

def combineA(mainSeries,total,a,count_D,DTotal):
    LIR = mainSeries[(mainSeries.index.labels[1] == a)]
    D_Result = pd.Series()
    
    for b in count_D:#内循环整合Designer name，将Designer的total整合到原数据当中
        D_Result = D_Result.append(combineB(LIR,DTotal,a,b))
       
    LITotal = pd.Series([total[a]],index =["%s Total"% total.index.levels[0][a]])
    LIResult = D_Result.append(LITotal)
    return LIResult
def combineB(childSeries,total,a,b):
    DR = childSeries[(childSeries.index.labels[1] == a) & (childSeries.index.labels[2] == b)]
    DTotal = pd.Series([total[(total.index.labels[1] == a) & (total.index.labels[2] == b)][0]],index =["%s Total"% total.index.levels[2][b]])
    DResult = DR.append(DTotal)
    return DResult

def tableMaker(SeriesArray):
    table = pd.DataFrame(SeriesArray)
    table = table.T#转置表格，因为原table是反的
    
    return table
    

In [26]:
def galleryFinalize(i):
    index = indexA()#导入index
    rawData = cleanData(i)#清洗数据，根据client筛选数据
    FDE,PI,CTP,AF,CTASW,CMS = SeriesMakerSum(rawData,index)#获得各列的数据，此数据会用于计算总和
    LO = SeriesMakerMax(rawData,index)#LO属于特殊，需要用于计算MAX
    FDE_DT,PI_DT,CTP_DT,AF_DT,CTASW_DT,CMS_DT = galleryDesignerTotal(PI,CTP,AF,CTASW,CMS,FDE)#以各个line item的Designer name来计算总和
    FDE_LIT,PI_LIT,CTP_LIT,AF_LIT,CTASW_LIT,CMS_LIT =galleryLineItemTotal(PI,CTP,AF,CTASW,CMS,FDE)#以line item 为单位计算总和
    FDE_T,PI_T,CTP_T,AF_T,CTASW_T,CMS_T = galleryTotal(FDE,PI,CTP,AF,CTASW,CMS)#计算这个client的总和
    LO_DM,LO_LIM,LO_T = galleryMax(LO)#同理计算LO的数据
    count_LI,count_D = count(FDE)#随便使用任意一个列数据放进去，计算一个client 中有多少个line item， 一个line item 中有多少个designer
    LO_Result = pd.Series()
    PI_Result = pd.Series()
    CTP_Result = pd.Series()
    AF_Result = pd.Series()
    CTASW_Result = pd.Series()
    CMS_Result = pd.Series()
    FDE_Result= pd.Series()#初始化，创建Series
    for a in count_LI:#整合数据，外循环整合line item，将line item的total数据（看上面combineB的内循环）
        LO_Result = LO_Result.append(combineA(LO,LO_LIM,a,count_D,LO_DM))
        PI_Result = PI_Result.append(combineA(PI,PI_LIT,a,count_D,PI_DT))
        CTP_Result = CTP_Result.append(combineA(CTP,CTP_LIT,a,count_D,CTP_DT))
        AF_Result = AF_Result.append(combineA(AF,AF_LIT,a,count_D,AF_DT))
        CTASW_Result = CTASW_Result.append(combineA(CTASW,CTASW_LIT,a,count_D,CTASW_DT))
        CMS_Result = CMS_Result.append(combineA(CMS,CMS_LIT,a,count_D,CMS_DT))
        FDE_Result = FDE_Result.append(combineA(FDE,FDE_LIT,a,count_D,FDE_DT))
    LO_T = pd.Series([LO_T],index=["Grand Total"])#初始化Total的数据类型，准备把总total整合上去
    PI_T = pd.Series([PI_T],index=["Grand Total"]) 
    CTP_T = pd.Series([CTP_T],index=["Grand Total"]) 
    AF_T = pd.Series([AF_T],index=["Grand Total"])
    CTASW_T = pd.Series([CTASW_T],index=["Grand Total"])
    CMS_T = pd.Series([CMS_T],index=["Grand Total"])
    FDE_T = pd.Series([FDE_T],index=["Grand Total"])
    #把Grand Total整合到数据列上，格式完成
    LO_Result=LO_Result.append(LO_T)
    PI_Result=PI_Result.append(PI_T)
    CTP_Result=CTP_Result.append(CTP_T)
    AF_Result=AF_Result.append(AF_T)
    CTASW_Result=CTASW_Result.append(CTASW_T)
    CMS_Result=CMS_Result.append(CMS_T)
    FDE_Result=FDE_Result.append(FDE_T)
    #创建需要计算rate的列
    CTPR_Result = CTP_Result/PI_Result#计算Click through to product rate
    ASCTR_Result = CTASW_Result/CTP_Result# 计算Advertiser site CTR
    CTPR_Result = CTPR_Result.apply(lambda x: format(x, '.2%'))
    ASCTR_Result = ASCTR_Result.apply(lambda x: format(x, '.2%'))
    LO_Result  = LO_Result.apply(lambda x:format(x,',.0f'))
    PI_Result  = PI_Result.apply(lambda x:format(x,',.0f'))
    CTP_Result=CTP_Result.apply(lambda x:format(x,',.0f'))
    AF_Result=AF_Result.apply(lambda x:format(x,',.0f'))
    CTASW_Result=CTASW_Result.apply(lambda x:format(x,',.0f'))
    CMS_Result=CMS_Result.apply(lambda x:format(x,',.0f'))
    FDE_Result=FDE_Result.apply(lambda x:format(x,',.0f'))
    #整合成为Table
    table = pd.concat([LO_Result,PI_Result,CTP_Result,CTPR_Result,AF_Result,CTASW_Result,ASCTR_Result,CMS_Result,FDE_Result],axis=1)
    return table

In [32]:
for i in filter:
    if(i[0] != 'Rivini'):
        continue
    table = galleryFinalize(i)
    print(table)

                                                      0          1       2  \
(10128147, 01/02/17 - 31/01/18, Alyne by Rita V...  115    768,378   2,864   
Alyne by Rita Vinieris Total                        115    768,378   2,864   
(10128147, 01/02/17 - 31/01/18, Rivini by Rita ...   81    542,035   1,365   
Rivini by Rita Vinieris Total                        81    542,035   1,365   
10128147 Total                                      196  1,310,413   4,229   
(10171381, 01/02/18 - 31/01/19, Alyne by Rita V...  114    217,017   1,066   
(10171381, 01/02/18 - 31/01/19, Alyne by Rita V...  114    374,450   1,573   
(10171381, 01/02/18 - 31/01/19, Alyne by Rita V...  114    281,530   1,359   
(10171381, 01/02/18 - 31/01/19, Alyne by Rita V...  114    346,948   1,532   
(10171381, 01/02/18 - 31/01/19, Alyne by Rita V...  114    363,216   1,670   
(10171381, 01/02/18 - 31/01/19, Alyne by Rita V...  114    331,000   1,682   
(10171381, 01/02/18 - 31/01/19, Alyne by Rita V...  114    393,5

In [25]:
table

In [8]:
ASCTR_Result

(10128147, 01/02/17 - 31/01/18, Alyne by Rita Vinieris, 2018/01)     5.73%
Alyne by Rita Vinieris Total                                         5.73%
(10128147, 01/02/17 - 31/01/18, Rivini by Rita Vinieris, 2018/01)    5.86%
Rivini by Rita Vinieris Total                                        5.86%
10128147 Total                                                       5.77%
(10171381, 01/02/18 - 31/01/19, Alyne by Rita Vinieris, 2018/02)     8.16%
(10171381, 01/02/18 - 31/01/19, Alyne by Rita Vinieris, 2018/03)     7.31%
(10171381, 01/02/18 - 31/01/19, Alyne by Rita Vinieris, 2018/04)     6.62%
(10171381, 01/02/18 - 31/01/19, Alyne by Rita Vinieris, 2018/05)     6.40%
(10171381, 01/02/18 - 31/01/19, Alyne by Rita Vinieris, 2018/06)     5.87%
(10171381, 01/02/18 - 31/01/19, Alyne by Rita Vinieris, 2018/07)     7.02%
(10171381, 01/02/18 - 31/01/19, Alyne by Rita Vinieris, 2018/08)     6.73%
(10171381, 01/02/18 - 31/01/19, Alyne by Rita Vinieris, 2018/09)     7.55%
(10171381, 01/02/18 - 31/